In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
import cv2 as cv

from tqdm import tqdm 

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb, rgb2gray
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
    
path_train_images = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/DADOS Intermediários/U-net/train/images/'
#path_train_images = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/train/images_aimonino/'
#path_train_images = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/train/images_fetuga/'
#path_train_images = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/train/images_lunadei/'
#path_train_images = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/train/images_Ai_Fe_Lu/'

path_train_masks = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/DADOS Intermediários/U-net/train/mask/'

#path_test = "D:/Users/N30090/OneDrive/UFES-Elétrica/Implementação Browning Index/IDAT/BI_Malignas_BBOBMO_PWBHEPL/"

dim = 256

In [ ]:
#Train path images
name_images = []
for filename in os.listdir(path_train_images):
    img = imread(os.path.join(path_train_images,filename))
    if img is not None:
        name_images.append(filename)
num_images = len(name_images)
print(num_images)
X = []
for i in range (0,num_images): 
    img = imread(path_train_images+name_images[i])
    img = resize(img, (dim, dim))
    X.append(img)

#Train path masks
name_masks = []
for filename in os.listdir(path_train_masks):
    mask = imread(os.path.join(path_train_masks,filename))
    if mask is not None:
        name_masks.append(filename)
num_masks = len(name_masks)
print(num_masks)
y = []
for i in range (0,num_masks): 
    mas = imread(path_train_masks+name_masks[i])
    mas = resize(mas, (dim, dim))    
    y.append(mas)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
image_x = random.randint(0, len(X_train))
plt.axis("off")
imshow(X_train[image_x])
plt.show()
plt.axis("off")
imshow(np.squeeze(y_train[image_x]))
plt.show()

In [ ]:
num_classes = 1

inputs = tf.keras.layers.Input((dim, dim,3))

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
b1 = tf.keras.layers.BatchNormalization()(c1)
r1 = tf.keras.layers.ReLU()(b1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(r1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
b2 = tf.keras.layers.BatchNormalization()(c2)
r2 = tf.keras.layers.ReLU()(b2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(r2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
b3 = tf.keras.layers.BatchNormalization()(c3)
r3 = tf.keras.layers.ReLU()(b3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(r3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
b4 = tf.keras.layers.BatchNormalization()(c4)
r4 = tf.keras.layers.ReLU()(b4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(r4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
b5 = tf.keras.layers.BatchNormalization()(c5)
r5 = tf.keras.layers.ReLU()(b5)
c5 = tf.keras.layers.Dropout(0.3)(r5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6 = tf.keras.layers.BatchNormalization()(u6)
u6 = tf.keras.layers.ReLU()(u6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u6)
u7 = tf.keras.layers.concatenate([u7, c3])
u7 = tf.keras.layers.BatchNormalization()(u7)
u7 = tf.keras.layers.ReLU()(u7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u7)
u8 = tf.keras.layers.concatenate([u8, c2])
u8 = tf.keras.layers.BatchNormalization()(u8)
u8 = tf.keras.layers.ReLU()(u8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(u8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9 = tf.keras.layers.BatchNormalization()(u9)
u9 = tf.keras.layers.ReLU()(u9)

outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(u9)

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
from keras import backend as K
from sklearn.metrics import jaccard_score


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def jacc_m(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection+1.0)/(K.sum(y_true_f)+K.sum(y_pred_f)-intersection+1.0+K.epsilon())

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m,jacc_m]) #

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
batch = 5
lr = 0.001
epoc = 25
callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, monitor='loss'),
        #tf.keras.callbacks.TensorBoard(log_dir='logs')
            ]
tf.config.get_visible_devices()
K.set_value(model.optimizer.learning_rate, lr)
model.fit(x=np.array(X_train), y=np.array(y_train), validation_data=(np.array(X_test),np.array(y_test)), batch_size=batch, epochs=epoc, callbacks=callbacks)

In [ ]:
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

plt.figure()
plt.plot( loss, 'r', label='Training loss')
plt.plot( val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

accuracy = model.history.history['accuracy']
val_accuracy = model.history.history['val_accuracy']

plt.figure()
plt.plot( accuracy, 'r', label='Training accuracy')
plt.plot( val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

jaccard = model.history.history['jacc_m']
val_jaccard = model.history.history['val_jacc_m']

plt.figure()
plt.plot( jaccard, 'r', label='Training Jaccard Index')
plt.plot( val_jaccard, 'b', label='Validation Jaccard Index')
plt.title('Training and Validation Jaccard Index')
plt.xlabel('Epoch')
plt.ylabel('Jaccard Index Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

f1 = model.history.history['f1_m']
val_f1 = model.history.history['val_f1_m']

plt.figure()
plt.plot( f1, 'r', label='Training F1 Score')
plt.plot( val_f1, 'b', label='Validation F1 Score')
plt.title('Training and Validation F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input image', 'Ground Truth', 'Predicted mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.rcParams.update({'font.size': 12})
        plt.title(title[i])
        plt.imshow(display_list[i], cmap='gray')
        plt.axis('off')
        plt.show()
    


for i in range(0,len(X_test)):
        sample_image = X_test[i]
        sample_mask = y_test[i]
        prediction = model.predict(sample_image[tf.newaxis, ...])[0]
        predicted_mask = prediction > 0.5
        dice_unet = display([sample_image, sample_mask,predicted_mask])
    


In [ ]:
loss,accuracy,f1_m,jacc_m = model.evaluate(np.array(X_test),np.array(y_test), verbose=0) #
print('Loss: ',loss)
print('Accuracy: ',accuracy)
print('F1 Score: ',f1_m)
print('Jaccard Score: ',jacc_m)

In [ ]:
#path_test = "D:/Users/N30090/OneDrive/UFES-Elétrica/Implementação Browning Index/IDAT/BI_Benignas_BBOBMO_PWBHEPL/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Benignas_BBOBMO_PWBHEPL_Original/"
#path_test = "D:/Users/N30090/OneDrive/UFES-Elétrica/Implementação Browning Index/IDAT/BI_Malignas_BBOBMO_PWBHEPL/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Malignas_BBOBMO_PWBHEPL_Original/"

#path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Benignas_BBOBMO_PWBHEPL_Aimonino/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Benignas_BBOBMO_PWBHEPL_Aimonino/"
#path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Malignas_BBOBMO_PWBHEPL_Aimonino/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Malignas_BBOBMO_PWBHEPL_Aimonino/"

#path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Benignas_BBOBMO_PWBHEPL_Fetuga/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Benignas_BBOBMO_PWBHEPL_Fetuga/"
#path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Malignas_BBOBMO_PWBHEPL_Fetuga/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Malignas_BBOBMO_PWBHEPL_Fetuga/"

#path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Benignas_BBOBMO_PWBHEPL_Lunadei/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Benignas_BBOBMO_PWBHEPL_Lunadei/"
#path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Malignas_BBOBMO_PWBHEPL_Lunadei/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Malignas_BBOBMO_PWBHEPL_Lunadei/"

#path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Benignas_BBOBMO_PWBHEPL_Aimonino+Fetuga+Lunadei/"
#path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Benignas_BBOBMO_PWBHEPL_Aimonino+Fetuga+Lunadei/"
path_test = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/BI_Malignas_BBOBMO_PWBHEPL_Aimonino+Fetuga+Lunadei/"
path_out = "D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/BI_Malignas_BBOBMO_PWBHEPL_Aimonino+Fetuga+Lunadei/"

images = []
for filename in os.listdir(path_test):
    img = imread(os.path.join(path_test,filename))
    if img is not None:
        images.append(filename)
num = len(images)
print(num)

for i in range (0,num): 
    img = imread(path_test+images[i])
    img = resize(img, (dim, dim))
    img = img[:,:,:3]
    #img = np.uint8(img)
    prediction = model.predict(img[tf.newaxis, ...])[0]
    predicted_mask = np.uint8(prediction > 0.5)
    im_rgb = cv.bitwise_and(img, img, mask=predicted_mask)
    plt.imshow(im_rgb)
    #pasta output
    q = path_out+images[i]
    #Salvando em PNG
    plt.imsave(q, im_rgb) 